Καμάρης Άγγελος
sdi1900070

**Τεχνητή Νοημοσύνη ΙΙ**

(ακαδ. έτος 2022-23)

Άσκηση 1:

In [ ]:
#headers used throughout the programm
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from nltk.tokenize import RegexpTokenizer
from nltk.probability import FreqDist
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_curve
import nltk
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
from nltk.probability import FreqDist
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
!pip install transformers
import transformers
from transformers import BertTokenizer, BertModel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 22.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file=open('/content/drive/MyDrive/imdb-reviews.csv','r', encoding="utf-8")
df= pd.read_csv(file, sep='\t' , on_bad_lines='skip')       #open the file stored in the drive and seperate it by tabs
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
test_df = None

In [ ]:
def preprocess(df):       #This function 


  df['review'] = df['review'].astype(str).str.lower()     #make everything lowercase
  regexp = RegexpTokenizer('\w+')
  df['review']=df['review'].apply(regexp.tokenize)        #tokenize the reviews from strings to lists of words

  bigwords = df['review'].apply(lambda x: ' '.join([item for item in x if len(item)>2]))      #keep only words that contain more that 2 letters

  all_words = ' '.join([word for word in bigwords])      #merge them and tokenize them again to create a frequency dictionary
  tw = nltk.tokenize.word_tokenize(all_words)

  fdist = FreqDist(tw)


  df['review'] = df['review'].apply(lambda x: ' '.join([item for item in x if fdist[item] >= 1 ]))  #remove not frequent words


  lem = WordNetLemmatizer()

  df['review'] = df['review'].apply(lem.lemmatize)                  #lematize



  
preprocess(df)

if test_df != None:                     #if a testset is given, preprocess it
  preprocess(test_df)

In [ ]:
smaller_df=df.iloc[:4500, :]
X=smaller_df['review']
Y =smaller_df[['rating']] # [[ ]] used to keep the label
Y.loc[Y['rating'] <= 4, 'rating'] = 0                 #keep only valuable data, for X and Y
Y.loc[Y['rating'] >= 7, 'rating'] = 1

if test_df == None:
  X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)              #if we are not given a test set, we will split our current data.
else:
  X_train = X['review']
  Y_train = Y
  X_test = test_df.drop('rating', axis=1)
  X_test = test_df.drop('url', axis=1)
  Y_test = test_df[['rating']] # [[ ]] used to keep the label

  Y_test.loc[Y_test['rating'] <= 4, 'rating'] = 0
  Y_test.loc[Y_test['rating'] >= 7, 'rating'] = 1


/usr/local/lib/python3.9/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.9/dist-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [ ]:
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from transformers import BertTokenizer
from torch.nn.utils.rnn import pad_sequence
from torch.nn.utils.rnn import pad_sequence


Y_train_tor=torch.tensor(np.array(Y_train), dtype=torch.long)
Y_test_tor=torch.tensor(np.array(Y_test), dtype=torch.long)
X_train_flat =X_train
X_test_flat =X_test


# Define your neural network architecture
# Load pre-trained BERT model and set it for fine-tuning
model = transformers.BertForSequenceClassification.from_pretrained('bert-base-uncased')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Prepare data for fine-tuning
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')




encoded_inputs_train = tokenizer.batch_encode_plus(X_train_flat,
                                                  add_special_tokens=True,
                                                  return_attention_mask=True,
                                                  pad_to_max_length=True,
                                                  max_length=90,
                                                  return_tensors='pt'
                                                   )
encoded_inputs_test = tokenizer.batch_encode_plus(X_test_flat,
                                                  add_special_tokens=True,
                                                  return_attention_mask=True,
                                                  pad_to_max_length=True,
                                                  max_length=90,
                                                  return_tensors='pt'
                                                  )

train_dataset = torch.utils.data.TensorDataset(encoded_inputs_train['input_ids'], encoded_inputs_train['attention_mask'], Y_train_tor)
test_dataset = torch.utils.data.TensorDataset(encoded_inputs_test['input_ids'], encoded_inputs_test['attention_mask'], Y_test_tor)

# Define the optimizer and the loss function
optimizer = transformers.AdamW(model.parameters(), lr=1e-5)
criterion = torch.nn.CrossEntropyLoss()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
val_labels=[]
val_preds=[]
def train_model(model, train_dataset, test_dataset, optimizer, criterion,val_labels ,val_preds ,num_epochs=5, batch_size=32):
    # Define data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Train the model
    for epoch in range(num_epochs):
        # Set model to training mode
        model.train()

        # Iterate over the training dataset
        for input_ids, attention_mask, labels in train_loader:
            # Move tensors to the device
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)

            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

        # Evaluate the model on the test dataset
        model.eval()
        test_loss = 0.0
        num_correct = 0
        num_total = 0
        for input_ids, attention_mask, labels in test_loader:
            # Move tensors to the device
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)

            # Forward pass
            with torch.no_grad():
                outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                logits = outputs.logits

            # Compute the test loss
            test_loss += loss.item() * labels.size(0)

            # Compute the number of correct predictions
            _, predicted = torch.max(logits, dim=1)
            num_correct += (predicted == labels).sum().item()
            num_total += labels.size(0)
            val_labels.extend(labels.cpu().numpy())
            val_preds.extend(predicted.cpu().numpy())

        test_loss /= len(test_dataset)
        test_accuracy = num_correct / num_total

        # Print the results for this epoch
        print('Epoch {}/{} - loss: {:.4f} - test_loss: {:.4f} - test_accuracy: {:.4f}'.format(epoch+1, num_epochs, loss.item(), test_loss, test_accuracy))
train_model(model, train_dataset, test_dataset, optimizer, criterion,val_labels ,val_preds, num_epochs=2, batch_size=48)

Epoch 1/2 - loss: 0.4365 - test_loss: 0.4445 - test_accuracy: 23.8644
Epoch 2/2 - loss: 0.4220 - test_loss: 0.3470 - test_accuracy: 24.1933


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print(accuracy_score(val_labels, val_preds))
print(precision_score(val_labels, val_preds))
print(recall_score(val_labels, val_preds))
print(f1_score(val_labels, val_preds))

0.825
0.8954489544895449
0.7599164926931107
0.8221343873517787
